In [1]:
import os

import numpy as np
from sklearn.decomposition import PCA

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [2]:
# params
log_dir = "/logfiles"
model = "bigscience-T0_3B" # bigscience-T0_B or bigscience-T0
module = "encoder" # encoder
# task = "rte"
task = "cb"
# task = "wic"

In [10]:
assert module == "encoder" # TODO(mm): support decoder as well

## Prompts

In [3]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}.csv")
display(df)

,name,template,category,includes_labels,shuffle
0,null_pattern,{premise} {hypothesis},neutral,False,False
1,null_pattern_reversed,{hypothesis} {premise},neutral,False,False
2,gpt_3_true_false_neither,"{premise} Question: {hypothesis} True, False, ...",instructive,True,False
3,gpt_3_yes_no_maybe,"{premise} Question: {hypothesis} Yes, No, or M...",instructive,True,False
4,mnli_crowdsource,{premise} Using only the above description and...,instructive,True,False
5,always_sometimes_never,"Suppose it's true that {premise} Then, is ""{hy...",instructive,True,False
6,based_on_previous_passage,"{premise} Based on the previous passage, is it...",instructive,True,False
7,infer,"Suppose {premise} Can we infer that ""{hypothes...",instructive,True,False
8,claim,"{premise} Based on that information, is the cl...",instructive,True,False
9,consider,"{premise} Keeping in mind the above text, cons...",instructive,True,False


## PCA

In [4]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [5]:
# RTE patterns
# use_pattern = [
#     "null_pattern",
#     "null_pattern_reversed",
#     "gpt_3_yes_no",
#     "gpt_3_yes_no_shuffled",
#     "gpt_3_true_false",
#     "gpt_3_true_false_shuffled",
#     "start_with_the",
#     "mnli_crowdsource",
#     "based_on_previous_passage",
#     "infer",
#     "follow",
#     "imply",
#     "guaranteed",
#     "justified", 
#     "must_be_true",
#     "should_assume"
# ]

# CB patterns
use_pattern = [
    "null_pattern",
    "null_pattern_reversed",
    "gpt_3_true_false_neither",
    "gpt_3_yes_no_maybe",
    "mnli_crowdsource",
    "always_sometimes_never",
    "based_on_previous_passage",
    "infer",
    "claim",
    "consider",
    "follow",
    "imply",
    "guaranteed",
    "guaranteed_possible",
    "justified",
    "must_be_true",
    "should_assume",
    "take_the_following",
]

# WIC patterns
# use_pattern = [
#     "gpt_3",
#     "gpt_3_yes_no",
#     "affirmation",
#     "grammar_homework",
#     "polysemous",
#     "question_context",
#     "question_meaning",
#     "question_meaning_yes_no",
#     "same_sense",
#     "similar_sense",
#     "similar_sense_yes_no",
# ]

In [9]:
for layer in range(0, 10):
# for layer in range(0, 25):
# for layer in range(24, 25):
    print('layer=', layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            prompt_names.append(row['name'])


    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        classes += n_sequences * [idx] # assign representations to classes
    
    classes = np.asarray(classes)

    # shuffle representations and classes
    X, y = unison_shuffled_copies(representations, classes)
    print(X.shape, y.shape)

    # perform PCA on hidden representations
    print('PCA for prompts:', prompt_names)

    for n_components in range(1, 5):
        pca = PCA(n_components=n_components)
        pca.fit(X)

        # variance explained by each of the principal components
        print(f"model:{model}; module:{module}; layer:{layer}; n_components: {n_components}; variance explained: {pca.explained_variance_ratio_}")
        print(np.sum(pca.explained_variance_ratio_))
    print('\n')


layer= 0


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 5161.09it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 1; variance explained: [0.9334017]
0.9334017
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 2; variance explained: [0.93340164 0.01033186]
0.9437335
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 3; variance explained: [0.93340164 0.01033187 0.00871112]
0.9524446
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 4; variance explained: [0.93340164 0.01033188 0.00871113 0.00424156]
0.9566862


layer= 1


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 5182.38it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 1; variance explained: [0.26860443]
0.26860443
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 2; variance explained: [0.2686043  0.20742142]
0.47602573
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 3; variance explained: [0.26860407 0.20742129 0.09264082]
0.56866616
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 4; variance explained: [0.26860434 0.20742121 0.09264082 0.05665776]
0.62532413


layer= 2


...

(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:7; n_components: 1; variance explained: [0.99956095]
0.99956095
model:bigscience-T0_3B; module:encoder; layer:7; n_components: 2; variance explained: [9.9956095e-01 2.7372042e-04]
0.99983466
model:bigscience-T0_3B; module:encoder; layer:7; n_components: 3; variance explained: [9.9956095e-01 2.7372033e-04 3.6880487e-05]
0.99987155
model:bigscience-T0_3B; module:encoder; layer:7; n_components: 4; variance explained: [9.9956095e-01 2.7372033e-04 3.6880410e-05 2.4751727e-05]
0.9998963


layer= 8


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 4651.29it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:8; n_components: 1; variance explained: [0.9995339]
0.9995339
model:bigscience-T0_3B; module:encoder; layer:8; n_components: 2; variance explained: [9.9953389e-01 2.5841262e-04]
0.9997923
model:bigscience-T0_3B; module:encoder; layer:8; n_components: 3; variance explained: [9.9953389e-01 2.5841253e-04 5.1561612e-05]
0.99984384
model:bigscience-T0_3B; module:encoder; layer:8; n_components: 4; variance explained: [9.9953389e-01 2.5841244e-04 5.1561583e-05 2.8426562e-05]
0.99987227


layer= 9


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 5180.44it/s]


(1008, 2048) (1008,)
PCA for prompts: ['null_pattern', 'null_pattern_reversed', 'gpt_3_true_false_neither', 'gpt_3_yes_no_maybe', 'mnli_crowdsource', 'always_sometimes_never', 'based_on_previous_passage', 'infer', 'claim', 'consider', 'follow', 'imply', 'guaranteed', 'guaranteed_possible', 'justified', 'must_be_true', 'should_assume', 'take_the_following']
model:bigscience-T0_3B; module:encoder; layer:9; n_components: 1; variance explained: [0.99948126]
0.99948126
model:bigscience-T0_3B; module:encoder; layer:9; n_components: 2; variance explained: [9.9948126e-01 2.5689061e-04]
0.99973816
model:bigscience-T0_3B; module:encoder; layer:9; n_components: 3; variance explained: [9.9948114e-01 2.5689069e-04 6.2855164e-05]
0.9998009
model:bigscience-T0_3B; module:encoder; layer:9; n_components: 4; variance explained: [9.9948114e-01 2.5689078e-04 6.2855150e-05 3.3748031e-05]
0.99983466


